# **Introduction**
     
  In this notebook we will train NN using Encoder only Transformer architecture for text classification task!!

## 1. Load the Dataset

In [1]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 48.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requi

In [3]:
from datasets import load_dataset
raw_datasets = load_dataset("IMDB")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## 2. Explore Data

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
label_features = raw_datasets["train"].features["label"]
label_features

ClassLabel(names=['neg', 'pos'], id=None)

In [53]:
labels_names = ["Negative", "Positive"]
labels_names

['Negative', 'Positive']

In [51]:
text = raw_datasets["train"][0]["text"]
label = raw_datasets["train"][0]["label"]
print(f"Text is {text}")
print(f"Label is {label}")

Text is I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betw

## 3. Process the Data

In [12]:
# First getting small dataset so we can try our model on small dataset first
train_data = raw_datasets["train"].select(range(5000))
test_data = raw_datasets["test"].select(range(5000))

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [14]:
train_data

Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

In [15]:
def tokenize_data(examples):
    # Tokenize the text
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding=True)

    # Extract the labels
    labels = examples["label"]

    return {"input_ids": tokenized_inputs["input_ids"], "labels": labels}

# Tokenize the datasets
train_data_tokenized = train_data.map(
    tokenize_data,
    batched=True,
    remove_columns=train_data.column_names,
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [16]:
test_data_tokenized = test_data.map(
    tokenize_data,
    batched=True,
    remove_columns=test_data.column_names,
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [17]:
train_data_tokenized

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 5000
})

In [18]:
test_data_tokenized

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 5000
})

## 4. Data Collation

In [20]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 5. Let's Get Dataset in DataLoaders

In [21]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data_tokenized, batch_size=8, shuffle=True,
                               collate_fn=data_collator)
test_dataloader = DataLoader(test_data_tokenized, batch_size=8, shuffle=False,
                              collate_fn=data_collator)

In [22]:
len(test_dataloader)

625

## 6. Building Our Encoder Model

In [23]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert self.head_dim * heads == embed_size, "Embed size needs to be div by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split embedding into self.heads pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        energy = torch.einsum("nqhd, nkhd->nhqk", queries, keys)

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=3)

        out = torch.einsum("nhql, nlhd->nqhd", attention, values).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

class TransformerEncoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length, num_classes):
        super(TransformerEncoder, self).__init__()
        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length
        )
        self.fc_out = nn.Linear(embed_size, num_classes)
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def forward(self, src):
        src_mask = self.make_src_mask(src)
        enc_out = self.encoder(src, src_mask)
        # Use the output of the last token (CLS token) for classification
        out = self.fc_out(enc_out[:, 0, :])
        return out


In [24]:
src_vocab_size = tokenizer.vocab_size
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(device)
embed_size = 512
num_layers = 6
heads = 8
forward_expansion = 4
dropout = 0.1
max_length = 512
num_classes = 2  # Example for binary classification

model = TransformerEncoder(src_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length, num_classes).to(device)

out = model(x)
out

tensor([[ 0.0862,  0.5973],
        [-0.2986, -0.3918]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [25]:
torch.argmax(torch.softmax(out, dim=1), dim=1)

tensor([1, 0], device='cuda:0')

In [26]:
x.shape

torch.Size([2, 9])

In [27]:
print(model)

TransformerEncoder(
  (encoder): Encoder(
    (word_embedding): Embedding(30522, 512)
    (position_embedding): Embedding(512, 512)
    (layers): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): SelfAttention(
          (values): Linear(in_features=64, out_features=64, bias=False)
          (keys): Linear(in_features=64, out_features=64, bias=False)
          (queries): Linear(in_features=64, out_features=64, bias=False)
          (fc_out): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (feed_forward): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): ReLU()
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (fc_out): L

## 7. Training Our Model

In [28]:
optimizer = torch.optim.Adam(model.parameters(), 0.01)
loss_fn = torch.nn.CrossEntropyLoss()

In [29]:
for batch in train_dataloader:
    input_ids = batch['input_ids']
    labels = batch['labels']
    attention_mask = batch['attention_mask']

    # Now you can use input_ids, labels, and attention_mask as needed
    # print(input_ids)

In [30]:
from tqdm.auto import tqdm

epoch = 3

for epoch in tqdm(range(epoch)):

  ### Training

  model.train()
  train_loss, train_acc = 0, 0

  for batch in train_dataloader:
    input_ids = batch["input_ids"].to(device)
    labels = batch["labels"].to(device)

    # Forward Pass
    y_pred = model(input_ids)

    # Calculating loss
    loss = loss_fn(y_pred, labels)
    train_loss += loss.item()

    # Optimizing
    optimizer.zero_grad()

    # Backpropogation
    loss.backward()

    # Optimizer step
    optimizer.step()

    # Accuracy
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += torch.mean((y_pred_class == labels).float()).item()

  train_loss = train_loss / len(train_dataloader)
  train_acc = train_acc / len(train_dataloader)

  print(f"train accuracy is: {train_acc}")
  print(f"train loss is: {train_loss}")

  ### Testing

  model.eval()

  test_loss, test_acc = 0, 0

  with torch.inference_mode():
    for test_batch in test_dataloader:
      test_input_ids = test_batch["input_ids"].to(device)
      test_labels = test_batch["labels"].to(device)

      # Forward pass
      test_pred = model(input_ids)

      # Loss
      loss = loss_fn(test_pred, test_labels)
      test_loss += loss.item()

      # Accuracy
      test_pred_class = torch.argmax(torch.softmax(test_pred, dim=1), dim=1)
      test_acc += torch.mean((test_pred_class == labels).float()).item()

  test_loss = test_loss / len(test_dataloader)
  test_acc = test_acc / len(test_dataloader)

  print(f"test accuracy is: {test_acc}")
  print(f"test loss is: {test_loss}")


  0%|          | 0/3 [00:00<?, ?it/s]

train accuracy is: 0.9998
train loss is: 0.0005748486042022634
test accuracy is: 1.0
test loss is: 0.0
train accuracy is: 1.0
train loss is: 0.0
test accuracy is: 1.0
test loss is: 0.0
train accuracy is: 1.0
train loss is: 0.0
test accuracy is: 1.0
test loss is: 0.0


## 8. Testing the model

In [54]:
text = "This is good movie"

inputs = tokenizer.encode(text, return_tensors="pt").to(device)
encoded_ids = inputs[0].tolist()

model.eval()
with torch.no_grad():
  output = model(inputs)
  print(output)
  print(labels_names[torch.argmax(torch.softmax(output, dim=1), dim=1)])


tensor([[ 12.0786, -12.1121]], device='cuda:0')
Negative


In [49]:
label

0